In [ ]:
!pip uninstall dgl -y

In [ ]:
! git pull


In [ ]:
# Your code that installs packages goes here
! pip install torch
#! pip install dgl -f https://data.dgl.ai/wheels/repo.html
#! pip install dgl-cu101==0.6.1
! pip install  dgl -f https://data.dgl.ai/wheels/cu121/repo.html
! pip install pandas
! pip install pyyaml
! pip install pydantic

In [ ]:
print(torch.__version__)

In [ ]:
import torch
import dgl.nn as dglnn

In [ ]:
import dgl
import dgl.function as fn
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:

class RGCN(nn.Module):
    def __init__(self, in_feat, hidden_feat, out_feat, num_rels):
        super(RGCN, self).__init__()
        self.W = nn.ModuleList([
            nn.Linear(in_feat, hidden_feat) for _ in range(num_rels*2)
        ])
        self.loop_W = nn.Linear(in_feat, hidden_feat)
        self.out_W = nn.Linear(hidden_feat, out_feat)

    def forward(self, g, feats):
        g = g.local_var()
        g.ndata['h'] = feats
        
        # Apply relation-specific weights
        for i in range(g.number_of_relations()):
            g.apply_edges(fn.copy_u('h', 'msg'), etype=g.etypes[i])
            g.send_and_recv((g.edges(etypes=g.etypes[i], form='uv')[0],
                            g.edges(etypes=g.etypes[i], form='uv')[1]),
                            [fn.mean('msg', 'h')], 
                            [self.W[i]], 
                            g.etypes[i])
        
        # Self-loop
        loop_messages = self.loop_W(g.ndata['h'])
        g.ndata['h'] = g.ndata['h'] + loop_messages
        
        # Output projection
        output = self.out_W(g.ndata['h'])
        
        return output